In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import urllib.request
import os
import chromedriver_autoinstaller
from bs4 import BeautifulSoup
import requests
from functools import reduce
import numpy as np

최초 요청받은 내용은 아래와 같음

* 컬럼 결합 전처리

Gin

Liqueur	Liquor	Liquer	Bitters	Bitter	Sambuca

Juice	Beverage	Fruit	Fruit Juice

Cordial	Tea	Water	Coffee

Brandy	Schnapps

Rum

Whisky	Whiskey

Vodka

Wine	Fortified wine	Fortified Wine	Aperitif	aperitif

Beer	Stout

Soft Drink	Soda	Cider	Cola

Flower	Herb	Spice	

Garnish

Milk

Cream

Syrup

Mineral - Rimming으로 head 변경

Spirit	- 여기 내용 있는거 드랍

Mixer	Sauce	Mix		- 컬럼 드랍

별도 요청 없을 경우 좌측에 해당하는 내용으로 더해서 묶어주면 됨

* 단위 통일을 위한 전처리 - 원본 api에서 불러온 데이터 값이 난잡한 관계로 엑셀 작업으로 수동 처리

unit convert

1shot = 1.5 oz
1jiggers = 1.5 oz

1part = 1 oz

1cl = 0.338 oz
1ml = 0.338 oz

1tsp = 0.167 oz
1tbsp = 0.5 oz


1slice = 1oz
1small bottle = 10oz (300ml)

In [ ]:
# final = []
# for i in range(len(data['drinks'])):
#     d1 = data['drinks'][i]
#     res = [d1['strDrink']]
#     info = {}
#     for i in list(map(lambda x: x, range(1,16))):
#         try:
#             # ing_name = d1['strIngredient'+str(i)]
#             ing_type = requests.get(f"https://www.thecocktaildb.com/api/json/v1/1/search.php?i={d1['strIngredient' + str(i)]}").json()['ingredients'][0]['strType']
#             ing_vol = d1['strMeasure'+str(i)]
#             info[ing_type]=ing_vol
#         except TypeError:
#             pass
#     res.append(info)
#     final.append(res)

주조기법의 경우 문장 형태로 들어가있어 동사의 활용 형태를 확인 후 컬럼 제작

In [ ]:
import string
alphabet = list(string.ascii_lowercase)
num = list(map(str,list(map(lambda x:x,range(0,10)))))
search_index = alphabet + num

inst_list = [['Shake','shake','Shaking','shaking','Shaked','shaked','shaker','Shaker'],
['Stir','stir','stirr','Stired','stired','Stirring','stirring'],
['Float','float','Floating','floating','Floated','floated'],
['Frost','frost','Frosting','frosting','Frosted','frosted','Frozen','frozen'],
['Blend','blend','Blending','blending','Blended','blended'],
['Build','build','Building','building','Built','built',]]

In [ ]:
# final = {"name":[], "info":[]}
# for k in search_index:
#     response = requests.get(f"https://www.thecocktaildb.com/api/json/v1/1/search.php?f={k}")
#     data = response.json()
#     try:
#         for j in range(len(data['drinks'])):
#             d1 = data['drinks'][j]
#             res = [d1['strDrink']]
#             info = {}
#             for i in list(map(lambda x: x, range(1,16))):
#                 try:
#                     # ing_name = d1['strIngredient'+str(i)]
#                     ing_type = requests.get(f"https://www.thecocktaildb.com/api/json/v1/1/search.php?i={d1['strIngredient' + str(i)]}").json()['ingredients'][0]['strType']
#                     ing_vol = d1['strMeasure'+str(i)]
#                     info[ing_type]=ing_vol
#                 except TypeError:
#                     pass
#             res.append(info)
#             final["name"].append(res[0])
#             final["info"].append(res[1])
#     except TypeError:
#         pass

API 호출을 위해 알파벳, 숫자를 하나로 더해 search_index 변수에 할당

In [ ]:
temp_index=['a','b']
search_index = alphabet + num

In [ ]:
ingredients_info = {"name":[], "info":[]}
instructions = {"name":[],"Instuction":[],"Glass":[]}
for t_index in search_index:
    response = requests.get(f"https://www.thecocktaildb.com/api/json/v1/1/search.php?f={t_index}")
    data = response.json()

    try:
        for j in range(len(data['drinks'])):
            d1 = data['drinks'][j]
            ing_res = [d1['strDrink']]
            info = {}
            for i in list(map(lambda x: x, range(1,16))):
                try:
                    # ing_name = d1['strIngredient'+str(i)]
                    ing_type = requests.get(f"https://www.thecocktaildb.com/api/json/v1/1/search.php?i={d1['strIngredient' + str(i)]}").json()['ingredients'][0]['strType']
                    ing_vol = d1['strMeasure'+str(i)]
                    info[ing_type]=ing_vol
                except TypeError:
                    pass
            ing_res.append(info)
            ingredients_info["name"].append(ing_res[0])
            ingredients_info["info"].append(ing_res[1])



            glass = data['drinks'][j]['strGlass']
            inst_glass_res = [d1['strDrink']]
            temp={}
            for text in range(len(inst_list)):
                if [x for item in inst_list[text] for x in data['drinks'][j]['strInstructions'].split() if item in x] != []:
                    temp[inst_list[text][0]]=1
                else:
                    temp[inst_list[text][0]]=0
            inst_glass_res.append(temp)
            inst_glass_res.append(glass)
            instructions["name"].append(inst_glass_res[0])
            instructions["Instuction"].append(inst_glass_res[1])
            instructions["Glass"].append(inst_glass_res[2])


    except TypeError:
        pass

Instruction의 합을 컬럼으로 제작. 해당 값이 0 또는 2 이상일 경우 검토 필요

In [ ]:
df = pd.DataFrame(ingredients_info)
objs = [df, pd.DataFrame(df['info'].tolist()).iloc[:, :100]]
df2 = pd.concat(objs, axis=1).drop('info', axis=1)

inst_df = pd.DataFrame(instructions)
inst_df.head()
objs = [inst_df, pd.DataFrame(inst_df['Instuction'].tolist()).iloc[:, :10]]
df3 = pd.concat(objs, axis=1).drop('Instuction', axis=1)
df3['Inst_sum'] = df3['Shake']+ df3['Stir']+ df3['Float']+ df3['Frost']+ df3['Blend']+ df3['Build']

raw_final = df3.merge(df2,how='inner',on='name')

위에서 요청한 컬럼 결합 전처리 과정

재료 포함여부 특성상 null 값이 대부분이라 fillna('')로 1차적으로 걸러준 뒤 더해준다 (해당 과정을 수행하지 않을 경우 결합이 의도대로 되지 않는다)

In [ ]:
final_df_name = raw_final['name'].fillna('').map(str)
final_df_Gin = raw_final['Gin'].fillna('').map(str)
final_df_Liqueur = raw_final['Liqueur'].fillna('').map(str) +  raw_final['Liquor'].fillna('').map(str) + raw_final['Liquer'].fillna('').map(str) + raw_final['Bitters'].fillna('').map(str) + raw_final['Bitter'].fillna('').map(str)+  raw_final['Sambuca'].fillna('').map(str)
final_df_Juice = raw_final['Juice'].fillna('').map(str) + raw_final['Beverage'].fillna('').map(str) +raw_final['Fruit'].fillna('').map(str) +raw_final['Fruit Juice'].fillna('').map(str)
final_df_Cordial = raw_final['Cordial'].fillna('').map(str) + raw_final['Tea'].fillna('').map(str) +raw_final['Water'].fillna('').map(str) +raw_final['Coffee'].fillna('').map(str)
final_df_Brandy = raw_final['Brandy'].fillna('').map(str) + raw_final['Schnapps'].fillna('').map(str)
final_df_Rum = raw_final['Rum'].fillna('').map(str)
final_df_Whisky = raw_final['Whisky'].fillna('').map(str) + raw_final['Whiskey'].fillna('').map(str)
final_df_Vodka = raw_final['Vodka'].fillna('').map(str)
final_df_Wine = raw_final['Wine'].fillna('').map(str) + raw_final['Fortified wine'].fillna('').map(str) +raw_final['Fortified Wine'].fillna('').map(str) +raw_final['Aperitif'].fillna('').map(str)+raw_final['aperitif'].fillna('').map(str)
final_df_Beer = raw_final['Beer'].fillna('').map(str) +  raw_final['Stout'].fillna('').map(str)
final_df_SoftDrink = raw_final['Soft Drink'].fillna('').map(str) +  raw_final['Soda'].fillna('').map(str) +  raw_final['Cider'].fillna('').map(str) +  raw_final['Cola'].fillna('').map(str)
final_df_Flower = raw_final['Flower'].fillna('').map(str) +  raw_final['Herb'].fillna('').map(str) + raw_final['Spice'].fillna('').map(str)
final_df_Garnish = raw_final['Garnish'].fillna('').map(str)
final_df_Milk = raw_final['Milk'].fillna('').map(str)
final_df_Cream = raw_final['Cream'].fillna('').map(str)
final_df_Syrup = raw_final['Syrup'].fillna('').map(str)
final_df_Rimming = raw_final['Mineral'].fillna('').map(str)
final_df_Spirit = raw_final['Spirit'].fillna('').map(str)

df4_list =  [final_df_name,final_df_Gin,final_df_Liqueur,final_df_Juice,final_df_Cordial,final_df_Brandy,final_df_Rum,final_df_Whisky,final_df_Vodka,final_df_Wine,final_df_Beer,final_df_SoftDrink,final_df_Flower,final_df_Garnish,final_df_Milk,final_df_Cream,final_df_Syrup,final_df_Rimming,final_df_Spirit]

In [ ]:
df4_data = {'name':df4_list[0], 'Gin':df4_list[1],'Liqueur':df4_list[2],'Juice':df4_list[3],
'Cordial':df4_list[4],'Brandy':df4_list[5],'Rum':df4_list[6],'Whisky':df4_list[7],'Vodka':df4_list[8],
'Wine':df4_list[9],'Beer':df4_list[10],'Soft Drink':df4_list[11],'Flower':df4_list[12],'Garnish':df4_list[13],
'Milk':df4_list[14],'Cream':df4_list[15],'Syrup':df4_list[16],'Mineral':df4_list[17], 'Spirit':df4_list[18]}

공백인 결측치를 연산이 가능하도록 0으로 대치한다

In [ ]:
df4 = pd.DataFrame(data=df4_data).replace('',0)

In [ ]:
df4

In [ ]:
modified_final = df3.merge(df4,how='inner',on='name')
modified_final.head()

요청받은 대로 Spirit을 drop 후 초안 csv 생성

In [ ]:
drop_idx = modified_final[modified_final['Spirit']!=0].index
modified_final.drop(drop_idx, inplace=True)

In [ ]:
drop_modified_final = modified_final.reset_index()
drop_modified_final

In [ ]:
modified_final.to_csv('cktl_modified.csv',sep=',',index=True)

In [ ]:
raw_final.to_csv('cktl_raw.csv',sep=',',index=True)

In [ ]:
drop_modified_final.to_csv('cktl_modified_dropsprit.csv',sep=',',index=True)

In [ ]:
# df4_list_01 = final_df_name
# df4_list_02 = final_df_Gin
# df4_list_03 = final_df_Liqueur
# df4_list_04 = final_df_Juice
# df4_list_05 = final_df_Cordial
# df4_list_06 = final_df_Brandy
# df4_list_07 = final_df_Rum
# df4_list_08 = final_df_Whisky
# df4_list_09 = final_df_Vodka
# df4_list_10 = final_df_Wine
# df4_list_11 = final_df_Beer
# df4_list_12 = final_df_SoftDrink
# df4_list_13 = final_df_Flower
# df4_list_14 = final_df_Garnish
# df4_list_15 = final_df_Milk
# df4_list_16 = final_df_Cream
# df4_list_17 = final_df_Syrup
# df4_list_18 = final_df_Rimming

In [ ]:
ing_only = pd.read_csv('cktl_modified_dropsprit.csv',encoding='cp949')

추가적으로 요청받은 내용

각 칵테일 별 재료만 추출해서 데이터프레임 제작

In [ ]:
# 필요없는 부분 drop 후 melt로 형태 변환
ing_only_df = ing_only.drop(['Glass','Shake','Stir','Float','Frost','Blend','Build','Inst_sum'],axis=1).melt(id_vars=["index", "name"], 
        var_name="Ingredient", 
        value_name="Volume")

In [ ]:
drop_idx2a = ing_only_df[ing_only_df['Volume']== 0].index
drop_idx2b = ing_only_df[ing_only_df['Volume']== "0"].index
drop_idx2 = drop_idx2a.append(drop_idx2b)
ing_only_df.drop(drop_idx2, inplace=True)

In [ ]:
ing_only_df

In [66]:
ing_only_df.to_csv('cktl_ing_only.csv',sep=',',index=True)


추가적으로 요청받은 dataframe - 알콜 포함여부

별도 지표가 없어 각종 베이스 기반 포함여부를 바탕으로 데이터프레임 제작

In [67]:
alc_df = pd.read_csv('cktl_modified_dropsprit.csv',encoding='cp949')

In [68]:
alc_df['Gin'].astype('float')
alc_df['Liqueur'].astype('float')
alc_df['Brandy'].astype('float')
alc_df['Rum'].astype('float')
alc_df['Whisky'].astype('float')
alc_df['Wine'].astype('float')
alc_df['Vodka'].astype('float')
alc_df['Beer'].astype('float')


0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
386    0.0
387    0.0
388    0.0
389    0.0
390    0.0
Name: Beer, Length: 391, dtype: float64

In [69]:
# np.where로 다중조건 탐색
alc_df['Alcoholic'] = np.where(
(alc_df['Gin']!= 0)|
(alc_df['Brandy']!= 0)|
(alc_df['Rum']!= 0)|
(alc_df['Whisky']!= 0)|
(alc_df['Wine']!= 0)|
(alc_df['Vodka']!= 0)|
(alc_df['Beer']!= 0), 1,0)

In [71]:
alc_df.head()
alc_df.to_csv('cktl_Alcoholic_appended.csv',sep=',')

In [ ]:
drop_idx2a = alc_df[alc_df['Volume']== 0, alc_df['Volume']=="0"].index
drop_idx2b = alc_df[alc_df['Volume']== "0"].index